In [23]:
import torch

s_data = torch.randn(64,3,10)

In [24]:
QWs = s_data.split(2, dim = -1)

In [34]:
QWs = torch.cat(QWs, dim = 0)

In [27]:
mask = torch.randn(64,3, 2)

# torch.cat([mask for _ in range(10)], dim = 0)

In [28]:
soft = torch.nn.Softmax(dim=-1)

In [29]:
soft(mask)[0,0,:]

tensor([0.4086, 0.5914])

In [41]:
torch.cat(QWs.split(64, dim = 0), dim = -1).shape

torch.Size([64, 3, 10])

In [40]:
QWs.shape

torch.Size([320, 3, 2])

# transformer.py, _generate_mask, mask.unsqueeze(1).expand(*x.size(), mask.size(-1))

In [122]:
a = torch.randn(4)
b = torch.randn(3)
c = torch.randn(2)

In [123]:
kk = torch.zeros(3,4)
kk[0,:len(a)] = a
kk[1,:len(b)] = b
kk[2,:len(c)] = c

In [124]:
@torch.no_grad()
def _generate_mask(x, length):
    '''

    에) length : [4,3,2]
    '''
    mask = []

    max_length = max(length)
    for l in length:
        if max_length - l > 0:
            # If the length is shorter than maximum length among samples,
            # set last few values to be 1s to remove attention weight.
            mask += [torch.cat([x.new_ones(1, l).zero_(),
                                x.new_ones(1, (max_length - l))
                                ], dim=-1)]
        else:
            # If length of sample equals to maximum length among samples,
            # set every value in mask to be 0.
            mask += [x.new_ones(1, l).zero_()]

    mask = torch.cat(mask, dim=0).bool()
    # |mask| = (batch_size, max_length)

    return mask

In [125]:
mask = _generate_mask(kk, [4,3,2])

In [128]:
mask

tensor([[False, False, False, False],
        [False, False, False,  True],
        [False, False,  True,  True]])

In [74]:
mask.shape

torch.Size([3, 4])

In [78]:
kk.size(), mask.size(-1)

(torch.Size([3, 4]), 4)

In [87]:
kk.shape

torch.Size([3, 4])

In [100]:
mask.unsqueeze(1).expand((3,4, 4))

tensor([[[False, False, False, False],
         [False, False, False, False],
         [False, False, False, False],
         [False, False, False, False]],

        [[False, False, False,  True],
         [False, False, False,  True],
         [False, False, False,  True],
         [False, False, False,  True]],

        [[False, False,  True,  True],
         [False, False,  True,  True],
         [False, False,  True,  True],
         [False, False,  True,  True]]])

In [88]:
mask.unsqueeze(1).shape

torch.Size([3, 1, 4])

In [102]:
@torch.no_grad()
def _generate_pos_enc(hidden_size, max_length):
    '''
    positioning encoding을 만들어 주는 함수.
    return [hidden_size * max_length]

    예) pos = 3(word) // dim_idx = 2 = 2*i

                    pos
        sin(  --------------- )
                10^4(2*i/d)

    '''
    enc = torch.FloatTensor(max_length, hidden_size).zero_()
    # |enc| = (max_length, hidden_size)

    pos = torch.arange(0, max_length).unsqueeze(-1).float()
    dim = torch.arange(0, hidden_size // 2).unsqueeze(0).float()
    # |pos| = (max_length, 1)
    # |dim| = (1, hidden_size // 2)

    enc[:, 0::2] = torch.sin(pos / 1e+4**dim.div(float(hidden_size)))
    enc[:, 1::2] = torch.cos(pos / 1e+4**dim.div(float(hidden_size)))

    return enc

In [104]:
_generate_pos_enc(100, 10).shape

torch.Size([10, 100])

In [115]:
from torch import nn

LS = nn.Softmax(dim=-1)

In [118]:
LS(torch.randn(10,20,500))[0][0].sum()

tensor(1.0000)

In [107]:
x = torch.randn(30,30)

In [110]:
torch.triu(x.new_ones(4,4))

tensor([[1., 1., 1., 1.],
        [0., 1., 1., 1.],
        [0., 0., 1., 1.],
        [0., 0., 0., 1.]])

In [133]:
aa = nn.Sequential(
    nn.Linear(400,200),
    nn.Linear(200,100),
    nn.Linear(100,50))

In [136]:
len(aa._modules)

3

In [140]:
for block in aa._modules.values():
    print(block)
    print('----------------')

Linear(in_features=400, out_features=200, bias=True)
----------------
Linear(in_features=200, out_features=100, bias=True)
----------------
Linear(in_features=100, out_features=50, bias=True)
----------------
